<a href="https://colab.research.google.com/github/pauloday/artbot/blob/fastrunner/Artbot_Studio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artbot Studio
by Paul O'Day

>Based on https://colab.research.google.com/drive/1L8oL-vLJXVcRzCFbPwOoMkPKJ8-aYdPN \

This is the runner for Artbot Studio, which will run in a different window. Here's how to run it:

1.   Go to the 'Runtime' menu, and select 'Change runtime type'. Make sure the accelerator is set to 'GPU'

2.   Run the init cell below by pressing the play button. You'll only have to run this once per session, even after you restart the runtime. It'll take a little while, it has to download at least 2GB of data to Google's servers, but the connection is really fast.

3.   After it's done downloading, run the next cell to upload your run file. You'll have to authenticate for Google Drive. Output will be saved in `Gaillery/`
4.   Have fun!


In [ ]:
print('Collecting init files...')
!git clone --branch fastrunner https://github.com/pauloday/artbot
!mv artbot/src/* .
print('Initializing...')
!./artbot/init.sh
!npm install -g localtunnel
import sys
sys.path.append('./taming-transformers')


fatal: destination path 'artbot' already exists and is not an empty directory.
mv: cannot stat 'artbot/src/artbot/*': No such file or directory
Initializing...
fatal: destination path 'CLIP' already exists and is not an empty directory.
fatal: destination path 'taming-transformers' already exists and is not an empty directory.
Obtaining file:///content/taming-transformers
  Attempting uninstall: taming-transformers
    Found existing installation: taming-transformers 0.0.1
    Can't uninstall 'taming-transformers'. No files were found to uninstall.
  Running setup.py develop for taming-transformers
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   645  100   645    0     0   1085      0 --:--:-- --:--:-- --:--:--  1085
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Curren

In [ ]:
from os import remove
from artbot import Artbot
from google.colab import files
from IPython import display
from tqdm.notebook import tqdm
from google.colab import drive
# comment this line out if you don't want google drive
drive.mount('/content/drive')

uploaded = files.upload()
image_pane = display.display(display_id=True)
status_pane = display.display(display_id=True)
def image_writer(img, video=False):
    if not video:
        image_pane.update(display.Image(img))
def status_writer(status):
    if not status:
        display.clear_output()
    else:
      status_pane.update(display.Markdown(status))

for fn in uploaded.keys():
  bot = Artbot(uploaded[fn], image_writer=image_writer, status_writer=status_writer, gallery='/content/drive/MyDrive/Gaillery', tqdm=tqdm)
  remove(fn)
  bot.run()

ImportError: ignored

The video writing progress bar only goes halfway and turns red, but it seems to write the whole video. Send any examples where that doesn't seem to be the case, or anything else interesting to artbot@pauloday.com.